# UI for RT1_Assignment3

In [1]:
# import
from IPython.display import display
from move_base_msgs.msg import MoveBaseActionGoal, MoveBaseGoal
from geometry_msgs.msg import Twist
import actionlib
from actionlib_msgs.msg import GoalID
import rospy
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, interact, interactive, fixed, interact_manual
from __future__ import print_function
from jupyros import ros3d 
import os

In [2]:
rospy.init_node('UI_node_jupyter') 

In [3]:
global pub
pub = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size=10)
global pubC
pubC = rospy.Publisher("/move_base/cancel", GoalID, queue_size=10)
global pubV
pubV = rospy.Publisher("/cmd_vel", Twist, queue_size=10)

vLin = 0.5
vAng = 1.0
vMaxLin = 2.0
vMaxAng = 4.0
vMinLin = 0.25
vMinAng = 0.5

## Menù

In [4]:
#### In this cell we define all the widgets used in this UI

In [5]:
# definition of the widgets for the position publication
xBox = widgets.BoundedFloatText(
    value=0.0,
    min=-10.00,
    max=10.0,
    step=1,
    description='X:',
    disabled=False
)
yBox = widgets.BoundedFloatText(
    value=0.0,
    min=-10.0,
    max=10.0,
    step=1,
    description='Y:',
    disabled=False
)
# definition of the buttons
bc = widgets.Button(
    description='Cancel the goal',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)
bt = widgets.Button(
    description='PUBLISH!',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)


# Buttons for the keyboard drive
b_front = widgets.Button(
    description='FRONT',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)
b_front_r = widgets.Button(
    description='FRONT-R',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)
b_front_l = widgets.Button(
    description='FRONT-L',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)
b_stop = widgets.Button(
    description='STOP',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)
b_turn_l = widgets.Button(
    description='TURN-L',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)
b_turn_r = widgets.Button(
    description='TURN-R',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)
b_decrease = widgets.Button(
    description='DEC. VEL',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)
b_increase = widgets.Button(
    description='INC. VEL',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)
b_back = widgets.Button(
    description='BACK',
    disabled=False,
    button_style='',
    tootltip='Click me',
    icon=''
)

# Definition of the rows to show
first_row=widgets.HBox([b_front_l, b_front, b_front_r])
second_row=widgets.HBox([b_turn_l, b_stop, b_turn_r])
third_row=widgets.HBox([b_back, b_increase, b_decrease])

In [6]:
#### Here we define all the functions used for the widgets

In [7]:
# Functions for the enable buttons
def enable_goal():
    publication_goal = True
    publication_cancel = False
    publication_keyboard = False

def enable_cancel():
    publication_goal = False
    publication_cancel = True
    publication_keyboard = False

def enable_keyboard():
    publication_goal = False
    publication_cancel = False
    publication_keyboard = True

# Functions for the click button of the keyboard
def front_vel(b_front):
    global vLin
    global pubV
    global v
    v = Twist()
    v.linear.x = vLin
    v.angular.z = 0.0
    pubV.publish(v)
    
def front_r_vel(b_front_r):
    global vLin
    global vAng
    global v
    v = Twist()
    v.linear.x = vLin
    v.angular.z = -vAng
    pubV.publish(v)
    
def front_l_vel(b_front_l):
    global vLin
    global vAng
    global v
    v = Twist()
    v.linear.x = vLin
    v.angular.z = vAng
    pubV.publish(v)
    
def turn_r_vel(b_turn_r):
    global vAng
    global v
    v = Twist()
    v.linear.x = 0.0
    v.angular.z = -vAng
    pubV.publish(v)
    
def turn_l_vel(b_turn_l):
    global vAng
    global v
    v = Twist()
    v.linear.x = 0.0
    v.angular.z = vAng
    pubV.publish(v)
    
def back_vel(b_back):
    global vLin
    global v
    v = Twist()
    v.linear.x = -vLin
    v.angular.z = 0.0
    pubV.publish(v)
    
def stop_vel(b_stop):
    global v
    v = Twist()
    v.linear.x = 0.0
    v.angular.z = 0.0
    pubV.publish(v)
    
def inc_vel(b_increase):
    global vLin
    global vAng
    global vMaxLin
    global vMaxAng
    global vMinLin
    global vMinAng
    global v
    v = Twist()
    if((vLin <= vMaxLin) and (vAng <= vMaxAng)):
        vLin += 0.1*vLin
        vAng += 0.1*vAng
    else:
        vLin = vMaxLin
        vAng = vMaxAng
        
    v.linear.x = Vlin
    v.angular.z = Vang
    
    pubV.publish(v)
    
def dec_vel(b_decrease):
    global vLin
    global vAng
    global vMaxLin
    global vMaxAng
    global vMinLin
    global vMinAng
    global v
    v = Twist()
    if((vLin >= vMinLin) and (vAng >= vMinAng)):
        vLin -= 0.1*vLin
        vAng -= 0.1*vAng
    else:
        vLin = vMinLin
        vAng = vMinAng
        
    v.linear.x = Vlin
    v.angular.z = Vang
    
    pubV.publish(v)
    
# Definition of the function for the button click
# Function of the button bt to publish the position
def pub_pos(bt):
    global g
    g = MoveBaseActionGoal()
    g.goal.target_pose.header.frame_id = "map"
    g.goal.target_pose.pose.position.x = xBox.value
    g.goal.target_pose.pose.position.y = yBox.value
    g.goal.target_pose.pose.orientation.w = 1.0
    g.goal_id.id = "g"
    pub.publish(g)
    
# Function of the button bc to cancel the goal
def bc_observer(bc):
    global gCanc
    global g
    gCanc = GoalID()
    if(g.goal_id.id == "g"):
        gCanc.id = g.goal_id.id
        pubC.publish(gCanc)
        g.goal_id.id = ""
        print("Goal cancelled successfully")
    else:
        print("No goal to cancel")

## Publish a (x,y) position

In [8]:
# Code fot the publication with the bt button
display(widgets.VBox([xBox, yBox]))
display(bt)
bt.on_click(pub_pos)

Button(description='PUBLISH!', style=ButtonStyle())

## Cancel the current goal

In [9]:
# Code to cancel the current goal set
display(bc)
bc.on_click(bc_observer)

Button(description='Cancel the goal', style=ButtonStyle())

Goal cancelled successfully


## Drive by keyboard

In [10]:
# Code to drive the robot by the keyboard use
display(widgets.VBox([first_row, second_row, third_row]))

b_front.on_click(front_vel)
b_front_r.on_click(front_r_vel)
b_front_l.on_click(front_l_vel)
b_turn_r.on_click(turn_r_vel)
b_turn_l.on_click(turn_l_vel)
b_back.on_click(back_vel)
b_stop.on_click(stop_vel)
b_increase.on_click(inc_vel)
b_decrease.on_click(dec_vel)

## Map

In [11]:
# Definition of the elements needed for the map
v = ros3d.Viewer()
rc = ros3d.ROSConnection(url="ws://localhost:9090") 
tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')
laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client) 
map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client) 
path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc, tf_client=tf_client)
urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client, path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000')) 
g = ros3d.GridModel()
v.objects = [g, laser_view, map_view, path, urdf]

# Map visualization
v

Viewer(objects=[GridModel(), LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_…